In [7]:
import pandas as pd
import calendar
import numpy as np
from sklearn.decomposition import PCA
from datetime import datetime



## Table 3 
hier komt tabel 3, maar ik was nog even bezig met het bekijken hoe zij deze data hebben verkregen

In [9]:
# loading dataframes 
r_daily = pd.read_csv('managed_portfolios_anom_d_50.csv')
# drop all momentum factors or factors that are constructed based on momentum

momentum_list = ['r_mom', 'r_indmom', 'r_valmom', 'r_valmomprof', 'r_mom12', 'r_momrev', 'r_indmomrev']
r_daily.drop(columns=momentum_list, inplace=True)

# set date to datetime format and set the date to the index 

r_daily['date'] = pd.to_datetime(r_daily['date'])
r_daily.set_index('date', inplace=True)

# missing list = ['dur', 'divg', 'invaci', 'ipo'] -> deze worden gemaakt uit daily stock data doormiddel van een signals list?

# following the procedure in the paper, if there are observations missing we set them to 0. 

r_daily.fillna(0, inplace=True)

# create a list of factors for later analysis purposes 

factors = [col for col in r_daily.columns if col.startswith('r_')]

In [10]:
# initialize pca model 

pca = PCA(n_components=len(factors))

# select our start date 

start_date = pd.to_datetime("1963-07-01")

# create an empty dataframe for our monthly pc returns

pc_return_df = pd.DataFrame(columns=['date'] + [f"PC{i+1}" for i in range(len(factors))])

# create our loop set up, this is actually an expanding PCA analysis. In each iteration a new month is added to the dataset and the return is computed. for year in range(1973, 202):
for year in range(1973, 2018):
    for mo in range(6, 13) if year == 1973 else range(1, 13):
        if year == 2017 and mo == 12:
            break
        else:            
            # first we have to find the last day of the month. For this we use the calender function with inputs from the loop variables
            last_day = calendar.monthrange(year, mo)[1]

            # we select our new end_date variable for which the PCA analysis is done, also with inputs from our loop and the last_day variable
            end_date = pd.to_datetime(f'{year}-{mo}-{last_day}')

            # we select the datarange from our dataset (July 1963 = start_date until our defined end_date) and we fit the model
            pca_data = r_daily.loc[start_date:end_date]
            pca.fit(pca_data)

            # we extract the principal components. These principal components are put in a new dataframe for later analysis. 

            principal_components = pca.transform(pca_data)
            principal_df = pd.DataFrame(data=principal_components, index=pca_data.index, columns=[f"PC{i+1}" for i in range(len(factors))])
            principal_df.index = pd.to_datetime(principal_df.index)

            # calculating return for month t+1. If mo = 12, then year will increment with 1. 

            return_year = year + 1 if mo == 12 else year
            return_month = (mo % 12) + 1

            # creating a datetime variable for the month t+1 and storing this in our pc_return_data variable

            return_date = pd.to_datetime(f'{return_year}-{return_month}')
            pc_return_data = {'date': return_date}

            # in this loop we calculate the monthly factor returns (f) using the principal components and returns

            for f in range(len(factors)):
                # select our factor and extract its principal component from principal_df and its return from r_daily for all observations in month mo 
                pc_f = principal_df[(principal_df.index.year == year) & (principal_df.index.month == mo)][principal_df.columns[f]]
                return_f = r_daily.loc[(r_daily.index.year == year) & (r_daily.index.month == mo), factors[f]]

                # multiply the principal components with the returns and sum them up to get PC factor return for month mo 
                pc_return_f = (pc_f*return_f).sum()

                # put the calculated return in our pc_return_data variable
                pc_return_data[principal_df.columns[f]] = pc_return_f

            # concat all the dataframes in our pc_return_df (this we can use for later analysis)
            pc_return_df = pd.concat([pc_return_df, pd.DataFrame(pc_return_data, index=[0])], ignore_index=True)

print(pc_return_df)

C:\Users\Tinot\AppData\Local\Temp\ipykernel_7632\2288442544.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pc_return_df = pd.concat([pc_return_df, pd.DataFrame(pc_return_data, index=[0])], ignore_index=True)


          date       PC1       PC2       PC3       PC4       PC5  \
0   1973-07-01  0.003799 -0.001069 -0.001030 -0.000200  0.000277   
1   1973-08-01 -0.000309 -0.002301 -0.002185 -0.000116  0.001866   
2   1973-09-01 -0.000313 -0.000372 -0.000095 -0.000378 -0.000045   
3   1973-10-01 -0.000108 -0.000733 -0.000717 -0.000208  0.000813   
4   1973-11-01 -0.000797 -0.000775  0.001822 -0.001357  0.000228   
..         ...       ...       ...       ...       ...       ...   
529 2017-08-01 -0.000628 -0.001236  0.000038 -0.000687  0.000473   
530 2017-09-01 -0.001829 -0.001535  0.000458 -0.000678  0.000294   
531 2017-10-01 -0.000909 -0.001662 -0.001341 -0.000121  0.001007   
532 2017-11-01 -0.000754 -0.000730  0.000688 -0.000286  0.000308   
533 2017-12-01 -0.001684 -0.001721 -0.001623  0.000178  0.000871   

              PC6       PC7       PC8       PC9  ...      PC34          PC35  \
0   -2.247953e-03 -0.000245  0.000516  0.000073  ...  0.000056  2.370831e-04   
1   -4.494985e-03  0.00

In [11]:
# Step 1: Compute individual factors' variances up to month t
individual_variances = pc_return_df.iloc[:, 1:].var()

# Step 2: Demean the PC factors
pc_return_df_demeaned = pc_return_df.copy()
pc_return_df_demeaned.iloc[:, 1:] -= pc_return_df.iloc[:, 1:].mean()

# Step 3: Lever the PC factors to match variances
pc_return_df_demeaned.iloc[:, 1:] *= np.sqrt(individual_variances / pc_return_df_demeaned.iloc[:, 1:].var())

# Step 4: Verify that variances and average returns are as specified
final_variances = pc_return_df_demeaned.iloc[:, 1:].var()
final_average_returns = pc_return_df_demeaned.iloc[:, 1:].mean()

# The final DataFrame with demeaned and levered PC factors is pc_return_df_demeaned

In [12]:
print(pc_return_df_demeaned)

          date       PC1       PC2       PC3       PC4       PC5       PC6  \
0   1973-07-01  0.001451 -0.001518 -0.000483  0.000955  0.000198 -0.001812   
1   1973-08-01 -0.002657 -0.002750 -0.001638  0.001039  0.001786 -0.004059   
2   1973-09-01 -0.002661 -0.000821  0.000453  0.000777 -0.000125 -0.000544   
3   1973-10-01 -0.002456 -0.001182 -0.000169  0.000947  0.000734 -0.000833   
4   1973-11-01 -0.003145 -0.001224  0.002369 -0.000202  0.000149 -0.002412   
..         ...       ...       ...       ...       ...       ...       ...   
529 2017-08-01 -0.002976 -0.001684  0.000585  0.000468  0.000394  0.000375   
530 2017-09-01 -0.004177 -0.001984  0.001006  0.000477  0.000215  0.000664   
531 2017-10-01 -0.003257 -0.002111 -0.000794  0.001034  0.000928  0.000563   
532 2017-11-01 -0.003102 -0.001179  0.001235  0.000869  0.000229  0.000436   
533 2017-12-01 -0.004032 -0.002170 -0.001076  0.001333  0.000791  0.000581   

          PC7       PC8       PC9  ...      PC34      PC35     

In [13]:
average_returns_list = []

# Set the lookback period (t)
lookback_period = 11

# Iterate through each row in the DataFrame starting from the 12th row
for i in range(lookback_period, len(pc_return_df_demeaned)):
    # Select the previous 11 observations for each factor
    lookback_data = pc_return_df_demeaned.iloc[i - lookback_period:i, 1:]

    # Calculate average returns over the lookback period
    average_returns_lookback = lookback_data.sum() / lookback_period

    # Append the average returns to the list
    average_returns_list.append(average_returns_lookback)

# Create a new DataFrame with the average returns
average_returns_df = pd.DataFrame(average_returns_list, columns=pc_return_df_demeaned.columns[1:])

# Add the 'date' column to the new DataFrame
average_returns_df['date'] = pc_return_df_demeaned['date'].iloc[lookback_period:].reset_index(drop=True)

# Reorder the columns to have 'date' as the first column
average_returns_df = average_returns_df[['date'] + [col for col in average_returns_df.columns if col != 'date']]

# Print the resulting DataFrame
print(average_returns_df)

          date       PC1       PC2       PC3       PC4       PC5       PC6  \
0   1974-06-01 -0.001250 -0.001688  0.000781  0.000759  0.002080 -0.003377   
1   1974-07-01 -0.001396 -0.001666  0.000838  0.000669  0.001800 -0.003521   
2   1974-08-01 -0.001175 -0.001517  0.000973  0.000600  0.001532 -0.003285   
3   1974-09-01 -0.000705 -0.001590  0.000744  0.000590  0.001512 -0.003218   
4   1974-10-01  0.000041 -0.001790  0.000718  0.000604  0.001348 -0.003214   
..         ...       ...       ...       ...       ...       ...       ...   
518 2017-08-01 -0.003367 -0.004541  0.000647 -0.000372  0.000961  0.000721   
519 2017-09-01 -0.003326 -0.004312  0.000598 -0.000213  0.001033  0.000695   
520 2017-10-01 -0.003501 -0.003954  0.000575 -0.000120  0.000968  0.000719   
521 2017-11-01 -0.003440 -0.003972  0.000437 -0.000072  0.001098  0.000670   
522 2017-12-01 -0.003583 -0.002962  0.000550  0.000153  0.000769  0.000702   

          PC7       PC8       PC9  ...      PC34      PC35     

In [18]:
# Assuming pc_return_df_demeaned is your DataFrame with demeaned returns
# and average_returns_df is your DataFrame with average returns

# Create a copy of pc_return_df_demeaned
new_returns_df = pc_return_df_demeaned.copy()

# Remove the first 12 rows
new_returns_df = new_returns_df.iloc[12:]

# Iterate through each row in average_returns_df
for i in range(len(average_returns_df) - 1):  # Exclude the last row as there's no next month for the last observation
    # Get the returns for the current month (t)
    returns_current_month_t = average_returns_df.iloc[i, 1:]

    # Identify factors with positive and negative returns
    positive_factors = returns_current_month_t[returns_current_month_t > 0].index
    negative_factors = returns_current_month_t[returns_current_month_t < 0].index

    # Get the date for the current row
    date_t = average_returns_df.iloc[i]['date']

    # Get the index for the current date in pc_return_df_demeaned
    index_t = new_returns_df.index[new_returns_df['date'] == date_t + pd.DateOffset(months=1)].tolist()

    if len(index_t) > 0:  # Check if there is a matching date
        index_t = index_t[0]
        # Set the returns for negative factors to 0 with a lag of one month
        new_returns_df.loc[index_t, negative_factors] = 0

# Reset the index to start from 0
new_returns_df.reset_index(drop=True, inplace=True)

# Print the resulting DataFrame
print(new_returns_df)

          date       PC1  PC2       PC3       PC4       PC5       PC6  \
0   1974-07-01  0.000000  0.0 -0.000160  0.000275 -0.001160  0.000000   
1   1974-08-01  0.000000  0.0 -0.002065  0.000673 -0.000343  0.000000   
2   1974-09-01  0.000000  0.0 -0.000447  0.001100 -0.001067  0.000000   
3   1974-10-01  0.000000  0.0 -0.001561  0.000143 -0.001101  0.000000   
4   1974-11-01  0.019224  0.0 -0.001975 -0.003475 -0.001129  0.000000   
..         ...       ...  ...       ...       ...       ...       ...   
517 2017-08-01  0.000000  0.0  0.000585  0.000000  0.000394  0.000375   
518 2017-09-01  0.000000  0.0  0.001006  0.000000  0.000215  0.000664   
519 2017-10-01  0.000000  0.0 -0.000794  0.000000  0.000928  0.000563   
520 2017-11-01  0.000000  0.0  0.001235  0.000000  0.000229  0.000436   
521 2017-12-01  0.000000  0.0 -0.001076  0.000000  0.000791  0.000581   

          PC7       PC8       PC9  ...      PC34  PC35      PC36      PC37  \
0    0.000000  0.000000  0.000000  ...  0.000

Als het goed is dit nu de momentum strategy waarbij factors met een gemiddelde positive return over de periode t-11 tot t, long zijn en factors met negatieve returns zijn short. Vervolgens zijn de returns berekend voor t+1. Dus alleen de factors met positive gemiddelde returns voor de periode t-11 tot t zitten in de portfolio.